# Reverse Logistics MVP

As a first step for my Reverse Logistics project, I will create a MVP which will show the trade lanes where this model can be applied. It will only compare the Tender itself for one customer. 

In order to know if Reverse Logistics can be applied for this customer and its lanes, it has to fulfill the following:
     * Origin City = Destination City (i.eg Lane 1 has an origin A and Lane 2 has a destination A)
     * By Definition, Origin and Destination country has to be the same
     * Annual volume has to be higher than 54, ~1 containers per week in the matching lanes.

This first MVP will exclude close cities (Google Maps API) and multiple combinations

## 1. Data exploration

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [43]:
file_root = 'Reverse_Logistics_Template.xlsx'

In [44]:
tender = pd.read_excel(file_root)

In [45]:
tender.head()

,Customer,Lane_ID,Origin_Country,Destination_Country,Origin_Port,Destination_Port,Origin_City,Destination_City,Equipment_Type,Shipping_Terms,Commodity,Volume
0,CustomerB,1089,ID,CN,Makassar,Xingang/Tianjin,Makassar,Xingang,20' Standard Dry,Port to Port,Cocoa Liquor,60.0
1,CustomerB,1090,ID,CN,Makassar,Shanghai,Makassar,Shanghai,20' Standard Dry,Port to Port,Cocoa Liquor,140.0
2,CustomerB,1093,CN,TH,Shanghai,Bangkok (Pat/Lat Krabang/Laem Chabang),Shanghai,Bangkok,20' Reefer,Port to Port,Confectionary,10.0
3,CustomerB,1094,CN,TH,Shanghai,Bangkok (Pat/Lat Krabang/Laem Chabang),Shanghai,Bangkok,40' Reefer,Port to Port,Confectionary,70.0
4,CustomerB,1095,CN,VN,Shanghai,Cat Lai,Shanghai,Ho Chi Minh,20' Reefer,Port to Port,Confectionary,5.0


In [46]:
#This will show as the Exporter Countries, Mexico, Belgium and United States as Top3
top_exporters = tender.groupby('Origin_Country').sum()
top_exporters.sort_values('Volume', ascending=False)

,Volume
Origin_Country,
MX,59804.000000
BE,29005.000000
US,26361.182235
TH,16303.772828
ZA,12884.000000
FR,10292.000000
CN,9634.000000
IN,8497.000000
DE,6411.000000


In [47]:
#This will show as the Importers Countries, United States, Japan and UK as Top3
top_importers = tender.groupby('Destination_Country').sum()
top_importers.sort_values('Volume', ascending=False)

,Volume
Destination_Country,
US,44120.375000
AU,13826.531584
JP,12908.000000
GB,11445.527857
GT,10171.000000
PR,7376.000000
AE,6958.338415
CO,6124.000000
MX,5927.000000


## 2. Cleaning Lanes with <54 volume

In [48]:
tender_cleaned = tender[~(tender['Volume'] <54)]
tender_cleaned.head()

,Customer,Lane_ID,Origin_Country,Destination_Country,Origin_Port,Destination_Port,Origin_City,Destination_City,Equipment_Type,Shipping_Terms,Commodity,Volume
0,CustomerB,1089,ID,CN,Makassar,Xingang/Tianjin,Makassar,Xingang,20' Standard Dry,Port to Port,Cocoa Liquor,60.0
1,CustomerB,1090,ID,CN,Makassar,Shanghai,Makassar,Shanghai,20' Standard Dry,Port to Port,Cocoa Liquor,140.0
3,CustomerB,1094,CN,TH,Shanghai,Bangkok (Pat/Lat Krabang/Laem Chabang),Shanghai,Bangkok,40' Reefer,Port to Port,Confectionary,70.0
9,CustomerB,1100,CN,PH,Shanghai,Manila,Shanghai,Manila,40' Reefer,Port to Port,Confectionary,186.0
11,CustomerB,1102,CN,MY,Shanghai,Port Klang,Shanghai,Port Klang,40' Reefer,Port to Port,Confectionary,100.0


In [49]:
rows_removed = tender['Lane_ID'].count() - tender_cleaned['Lane_ID'].count()
rows_removed

2043

In [50]:
tender_cleaned['Lane_ID'].count()
#There are 2043 valid lanes to apply Reverse Logistics to its own Tender.

496

## 3. Matching the Trade Lanes

In [51]:
#Let's check if any city has more than 54 loads per year both as importer and exporter

In [52]:
city_exports = tender.groupby('Origin_City').sum()
city_exports.sort_values('Volume', ascending=False)

,Volume
Origin_City,
Bangkok,14450.772828
Merida Door,8384.000000
Nimes,3194.000000
Saint Petersburg,2981.000000
Veghel,2947.000000
Melbourne,2556.000000
Santiago Door,2393.000000
Shanghai,2346.000000
"Victorville, CA",2129.872706


In [53]:
city_exports = city_exports.rename({'Volume': 'Export_Volume'}, axis=1)

In [54]:
#Let's do the same for imports
city_imports = tender.groupby('Destination_City').sum()
city_imports.sort_values('Volume', ascending=False)
city_imports = city_imports.rename({'Volume': 'Import_Volume'}, axis=1)


In [55]:
#Let's Concatenate both tables:
total_volume = pd.concat([city_exports, city_imports], axis=1, join="inner")
total_volume.reset_index(level=0, inplace=True)
total_volume = total_volume.rename({'index': 'City'}, axis=1)
total_volume.head()

,City,Export_Volume,Import_Volume
0,Bangkok,14450.772828,147.000
1,Bangkok (Lat Krabang),30.000000,407.000
2,Brisbane,39.000000,374.000
3,Busan,386.000000,1824.734
4,"CHICAGO, IL",15.000000,15.000


In [56]:
#Let's see in which cities we can apply the reverse logistics (not taking into account the proximity of the cities, yet), for this they need to have at least 54 ctrs in both imports and exports
total_volume_cleaned = total_volume[~(total_volume['Export_Volume'] <54) & ~(total_volume['Import_Volume'] <54) ]

total_volume_cleaned.head()

,City,Export_Volume,Import_Volume
0,Bangkok,14450.772828,147.000000
3,Busan,386.000000,1824.734000
5,Davao,408.000000,119.000000
7,Durban,118.000000,1179.520833
9,"Greenville, MS",59.000000,104.000000


In [57]:
#Let's see all the lanes with Bangkok as Origin City by equipment type
Altamira_Export = tender[tender['Origin_City']=='Bangkok']
Altamira_Export.groupby('Equipment_Type')['Volume'].sum()

Equipment_Type
20' Reefer             15.000000
20' Standard Dry     1770.250000
40' High Cube       12637.522828
40' Reefer             28.000000
Name: Volume, dtype: float64

In [58]:
#Let's do the same by Destination
Altamira_Import = tender[tender['Destination_City']=='Bangkok']
Altamira_Import.groupby('Equipment_Type')['Volume'].sum()

Equipment_Type
20' Reefer    50.0
40' Reefer    97.0
Name: Volume, dtype: float64

## Conculsion

In this case, and under the criteria we use it would be difficult to apply the reverse logistics